### Loading and Merging ERA5 Climate Variables

#### Purpose
This script is designed to load and merge multiple ERA5 climate variables over a specified period. It retrieves data from various NetCDF files that contain monthly-averaged climate variables. The final output is a unified dataset (`merged_ds`) that contains all the specified variables (e.g., total precipitation, wind components, temperature) for the years 2020 and 2021. The dataset is constructed in a manner that enables easy analysis of these variables over time and across the spatial domain.

#### Process
1. **Define Inputs:**
   - The **ERA5 file path** (`era5_path`) is set to the directory containing the monthly-averaged data files.
   - The **variables** list contains the names of the climate variables to be loaded: total precipitation (`tp`), zonal wind component at 10m (`u10n`), meridional wind component at 10m (`v10n`), surface pressure (`sp`), and 2m air temperature (`2t`).
   - The **years** list is defined to include the years 2020 and 2021, specifying the time range for the data.
   - The **file pattern** (`file_pattern`) matches the ERA5 NetCDF files.

2. **Loading Variables:**
   - A function `load_era5_variable` is defined to load a single variable for the specified years. It looks for matching NetCDF files for each year and variable using `glob`. 
   - The function uses **lazy loading** (`xr.open_mfdataset`) to open the data, which ensures that only the necessary data is loaded into memory as needed, without pre-loading the entire dataset.

3. **Merging Datasets:**
   - The script then uses the `load_era5_variable` function to load the data for each variable in the list (`variables`).
   - All the individual datasets for the variables are merged using `xr.merge`. This ensures that they are combined based on common dimensions: **time**, **latitude**, and **longitude**.

4. **Final Dataset:**
   - The resulting merged dataset (`merged_ds`) contains all variables as data variables, with time, latitude, and longitude as coordinates. This unified dataset is ready for further analysis, such as calculating averages, performing statistical analysis, or visualizing the data.

The output of the script is the `merged_ds` dataset, which can be further used for processing or exported for additional analysis.


In [1]:
import xarray as xr
import os
from glob import glob

# Define inputs
era5_path = '/g/data/rt52/era5/single-levels/monthly-averaged'
variables = ['tp', 'u10n', 'v10n', 'sp', '2t']
years = ['2020', '2021']
file_pattern = '*era5_moda_sfc*.nc'

def load_era5_variable(var, years, base_path):
    """Load a single ERA5 variable across specified years."""
    file_paths = []
    for year in years:
        dir_path = os.path.join(base_path, var, year)
        file_paths.extend(glob(os.path.join(dir_path, file_pattern)))
    
    # Lazy load and concatenate along time
    ds = xr.open_mfdataset(file_paths, combine='by_coords', parallel=True)
    return ds

# Load and merge all variables
datasets = [load_era5_variable(var, years, era5_path) for var in variables]

# Merge all datasets on common dimensions (time, lat, lon)
merged_ds = xr.merge(datasets)

# Now you can work with `merged_ds`
merged_ds


<xarray.Dataset> Size: 997MB
Dimensions:    (time: 24, latitude: 721, longitude: 1440)
Coordinates:
  * longitude  (longitude) float32 6kB -180.0 -179.8 -179.5 ... 179.5 179.8
  * latitude   (latitude) float32 3kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 192B 2020-01-01 2020-02-01 ... 2021-12-01
Data variables:
    tp         (time, latitude, longitude) float64 199MB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    u10n       (time, latitude, longitude) float64 199MB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    v10n       (time, latitude, longitude) float64 199MB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    sp         (time, latitude, longitude) float64 199MB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    t2m        (time, latitude, longitude) float64 199MB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2020-09-14 11:22:07 UTC+1000 by era5_replication_tools-1.1....
    license:      Licence to use Copernicus Products: https://apps.ecmwf.int/...
    summary:      ERA5 is the fifth generation ECMWF atmospheric reanalysis o...
    title:        ERA5 single-levels monthly-averaged total_precipitation 202...

### Convert and Sort Longitude

This section of the script performs two tasks:

1. **Convert Longitude from [-180, 180] to [0, 360]:**
   - The longitude values in the dataset are transformed by adding 360 and taking the modulo 360. This ensures the longitudes fall within the range of 0 to 360 degrees.

2. **Sort Longitudes:**
   - The dataset is sorted by longitude in increasing order to ensure that the data is correctly ordered spatially along the longitude axis.

These steps help standardize and order the spatial coordinates for easier analysis and visualization.


In [2]:
# Convert longitude from [-180, 180] to [0, 360]
merged_ds = merged_ds.assign_coords(
    longitude=(((merged_ds.longitude + 360) % 360))
)

# Sort longitudes so the data is in increasing order
merged_ds = merged_ds.sortby('longitude')


### Select One Year
From March 2020 to February 2021, as agreed by the team.


In [3]:
# Time slice from 2020-03 to 2021-02 (inclusive)
merged_ds = merged_ds.sel(time=slice("2020-03", "2021-02"))


### Compute Seasonal and Total Averages

This step calculates the climatological means across time for the full ensemble dataset:

- **Total Average**: Mean over all time steps.
- **JJA**: June–August average (austral winter).
- **DJF**: December–February average (spanning years).
- **MAM**: March–May average (austral autumn).
- **SON**: September–November average (austral spring).

Each average is computed by selecting the relevant months and taking the mean along the `time` dimension.


In [4]:
# Total Time Average
total_avg = merged_ds.mean(dim='time')

# JJA (June, July, August) Average
jja_avg = merged_ds.sel(time=merged_ds['time.month'].isin([6, 7, 8])).mean(dim='time')

# DJF (December, January, February) Average
djf_avg = merged_ds.sel(time=merged_ds['time.month'].isin([12, 1, 2])).mean(dim='time')

# MAM (March, April, May) Average
mam_avg = merged_ds.sel(time=merged_ds['time.month'].isin([3, 4, 5])).mean(dim='time')

# SON (September, October, November) Average
son_avg = merged_ds.sel(time=merged_ds['time.month'].isin([9, 10, 11])).mean(dim='time')


### Estimate Memory Size of DJF Average

This block calculates the in-memory size of the `djf_avg` dataset in megabytes. While this provides a sense of the dataset's footprint during processing, note that saving the dataset with compression (e.g., NetCDF with `zlib=True`) will significantly reduce disk space usage.


In [5]:
# Calculate the size of djf_avg in bytes
djf_avg_size_bytes = djf_avg.nbytes

# Convert bytes to megabytes
djf_avg_size_mb = djf_avg_size_bytes / 1024 / 1024

# Print the result
print(f"Size of djf_avg: {djf_avg_size_mb:.2f} MB")


Size of djf_avg: 39.61 MB


### Save Averages to NetCDF Files

This step saves the computed seasonal and total averages to NetCDF files. The output is saved in the specified directory with filenames indicating the type of average (e.g., `tot`, `jja`, `djf`) and the corresponding period.

Compression is applied to reduce file size (`zlib=True`, `complevel=2`), ensuring efficient storage.

Files are saved with the following naming convention:
- `era5_tot_avg_202003_202102.nc`
- `era5_jja_avg_202003_202102.nc`
- `era5_djf_avg_202003_202102.nc`
- `era5_mam_avg_202003_202102.nc`
- `era5_son_avg_202003_202102.nc`


In [ ]:
# Output directory
record_path = '/scratch/nf33/hk25-ConvZones'
period_str = '202003_202102'

# Compression settings
encoding = {var: {'zlib': True, 'complevel': 2} for var in djf_avg.data_vars}

# Save Total Average
total_avg.to_netcdf(
    f'{record_path}/era5_tot_avg_{period_str}.nc',
    encoding=encoding
)

# Save JJA Average
jja_avg.to_netcdf(
    f'{record_path}/era5_jja_avg_{period_str}.nc',
    encoding=encoding
)

# Save DJF Average
djf_avg.to_netcdf(
    f'{record_path}/era5_djf_avg_{period_str}.nc',
    encoding=encoding
)

# Save JJA Average
mam_avg.to_netcdf(
    f'{record_path}/era5_mam_avg_{period_str}.nc',
    encoding=encoding
)

# Save DJF Average
son_avg.to_netcdf(
    f'{record_path}/era5_son_avg_{period_str}.nc',
    encoding=encoding
)
